In [1]:
import json
import jsonlines
import sys
import prompt_rag

search_engine = "brave"

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

import covmis


合并多个文件的先验知识

创建数据（带有先验知识的Prompt）以微调LLM

测试模型的先验知识生成效果：一次提问

In [2]:
import torch
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

dirs = ["../.."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    ModelType, get_vllm_engine, get_default_template_type,
    get_template, inference_vllm, VllmGenerationConfig, get_model_tokenizer
)
from custom import CustomModelType, CustomTemplateType

# model_type = ModelType.mixtral_moe_7b_instruct

# model_type = CustomModelType.phi_3_medium_4k_instruct
# model_type = CustomModelType.llama_3_70b_instruct_awq
model_type = CustomModelType.mixtral_moe_7b_instruct_awq
# model_type = CustomModelType.solar_instruct_10_7b

llm_engine = get_vllm_engine(
    model_type, 
    # torch_dtype=torch.float16,  # 检查正确的数据类型！！！！
    tensor_parallel_size=2,
    max_model_len=4096,
    # gpu_memory_utilization=0.92,
    # model_id_or_path="/home/css/models/Mixtral-8x7B-Instruct-v0.1-GPTQ-int4",
    engine_kwargs = {
        # "enforce_eager": True,
        "max_num_seqs": 1, # 64
        "seed": 42,
    }
)

_, tokenizer = get_model_tokenizer(
    model_type, 
    load_model=False
)

template_type = get_default_template_type(model_type)
template = get_template(template_type, llm_engine.hf_tokenizer)

generation_config = VllmGenerationConfig(
    max_new_tokens=2048,
    temperature=0,
)

get_resp_list = lambda request_list : inference_vllm(
    llm_engine, template, request_list, 
    generation_config=generation_config, 
    use_tqdm=True
)


2024-07-01 20:04:33,196 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-07-01 20:04:33,199 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-07-01 20:04:33,233 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 21bc0b9ccf26bcf3f4ca2e675ec8875d and a total number of 980 components indexed
[INFO:swift] Successfully registered `/home/hanlv/workspace/code/research/infodemic/LLM/swift/swift/llm/data/dataset_info.json`
[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  

WARNING 07-01 20:04:33 config.py:217] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.


2024-07-01 20:04:36,112	INFO worker.py:1753 -- Started a local Ray instance.


INFO 07-01 20:04:37 config.py:623] Defaulting to use mp for distributed inference
INFO 07-01 20:04:37 llm_engine.py:161] Initializing an LLM engine (v0.5.0.post1) with config: model='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=42, served_model_name=/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ)
INFO 07-01 20:04:37 selector.py:150] Cannot use FlashAttention-2 backend due to sliding window.
INFO 07-01 20:04:37 selector.py:51] Usin

(raylet) [2024-07-01 20:04:46,077 E 608521 608551] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_20-04-33_937911_607167 is over 95% full, available space: 15756500992; capacity: 1967317549056. Object creation will fail if spilling is required.


INFO 07-01 20:04:46 distributed_gpu_executor.py:56] # GPU blocks: 9029, # CPU blocks: 4096
INFO 07-01 20:04:48 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-01 20:04:48 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=612122) INFO 07-01 20:04:48 model_runner.py:889] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=612122) INFO 07-01 20:04:48 model_runner.py:893] CUDA graphs can take additional 1~3 GiB memory p

[INFO:swift] Loading the model using model_dir: /home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ
[INFO:swift] model_config: MixtralConfig {
  "_name_or_path": "/home/css/models/Mixtral-8x7B-Instruct-v0.1-AWQ",
  "architectures": [
    "MixtralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mixtral",
  "num_attention_heads": 32,
  "num_experts_per_tok": 2,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "num_local_experts": 8,
  "output_router_logits": false,
  "quantization_config": {
    "bits": 4,
    "group_size": 128,
    "modules_to_not_convert": [
      "gate"
    ],
    "quant_method": "awq",
    "version": "gemm",
    "zero_point": true
  },
  "rms_norm_eps": 1e-05,
  "rope_theta": 1000000.0,
  "router_aux_loss_coef": 0.02,
  "router_jitter_noise": 0.0,
  "sliding_window": 4

In [3]:
from tqdm.notebook import tqdm
with open(f"train_search_llm_tmp.json", "r") as f:
    prior_knowledge_list = json.load(f)
K = 5

n_truncate = 1000

model_name = 'mixtral'
data_train = covmis.load_train()
data_search = covmis.load_train_search()

update_list = []
id_list = []
prompt_list = []
for i in range(len(prior_knowledge_list)):
    item = prior_knowledge_list[i]

    p1 = item["prior_knowledge_mixtral"]

    if (len(tokenizer(p1)["input_ids"]) < 240 or len(tokenizer(p1)["input_ids"]) > 800):
        
        prompt = prompt_rag.get_prompt_for_generating_prior_knowledge(
            data_train[i]["claim"], data_train[i]["date"], 
            search_engine, data_search[i][f"{search_engine}_search_results"], 
            model_name, K=K, sort=False, ids=None, n_truncate=n_truncate
        )
        prompt_list.append(prompt)
        update_list.append(i)
        id_list.append(item["id"])

request_list = [{'query': prompt} for prompt in prompt_list]
resp_list = get_resp_list(request_list)
for i, resp in enumerate(resp_list):
    item = prior_knowledge_list[update_list[i]]
    if item["id"] != id_list[i]:
        raise Exception()
    item["prior_knowledge_mixtral"] = resp["response"].strip()
    _len = len(tokenizer(item["prior_knowledge_mixtral"])["input_ids"])
    if _len < 240 or \
        _len > 800:
        
        with open("error.txt", "a") as f:
            f.writelines(f"{update_list[i]}:{_len}")
            f.writelines("\n")

with open(f"train_search_llm_tmp.json", "w") as f:
    json.dump(prior_knowledge_list, f, indent=4)


(raylet) [2024-07-01 20:04:56,085 E 608521 608551] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_20-04-33_937911_607167 is over 95% full, available space: 15756419072; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-07-01 20:05:06,092 E 608521 608551] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_20-04-33_937911_607167 is over 95% full, available space: 15756414976; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-07-01 20:05:16,099 E 608521 608551] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_20-04-33_937911_607167 is over 95% full, available space: 15756406784; capacity: 1967317549056. Object creation will fail if spilling is required.
(raylet) [2024-07-01 20:05:26,106 E 608521 608551] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-01_20-04-33_937911_607167 is over 95% full, available space: 15756398592; capacity: 1967317549056.

100%|██████████| 1/1 [00:13<00:00, 13.90s/it]


In [4]:
print(prompt_list[0])

Below is some INFORMATION searched online and a CLAIM. These pieces of INFORMATION are relevant to the CLAIM. This CLAIM and all INFORMATION include their respective publication dates and contents. To classify the CLAIM more accurately (if the content described by the CLAIM is correct, it will be classified as TRUE; if the content described by the CLAIM is incorrect, it will be classified as FALSE), please first provide a clear summary of the given INFORMATION, and provide reasonable evidence to judge the correctness of the CLAIM based on the available information and your knowledge.

INFORMATION:
Information 1:
Publication date: 2020-05-19
Title: New study warns of COVID-19 impact on cardiovascular health
Content:
A recent study shows that a significant number of people who develop coronavirus disease 19 (COVID-19) may experience serious cardiovascular complications. Moreover, some possible treatments of the respiratory disease could end up harming cardiovascular health. ... A new stu

In [6]:
# data_search_llm = covmis.load_train_llm()
# print(data_search_llm[1096]["prior_knowledge_llama3_v1_K=5"])

for i, resp in enumerate(resp_list):
    item = prior_knowledge_list[update_list[i]]
    if item["id"] != id_list[i]:
        raise Exception()
    item["prior_knowledge_mixtral"] = resp["response"].strip()
    _len = len(tokenizer(item["prior_knowledge_mixtral"])["input_ids"])
    if _len < 240 or \
        _len > 800:
        print(item["prior_knowledge_mixtral"])
        print('\n\n\n')

(VllmWorkerProcess pid=612122) ERROR 07-01 20:35:47 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method start_worker_execution_loop: [../third_party/gloo/gloo/transport/tcp/unbound_buffer.cc:81] Timed out waiting 1800000ms for recv operation to complete, Traceback (most recent call last):
(VllmWorkerProcess pid=612122) ERROR 07-01 20:35:47 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/vllm/executor/multiproc_worker_utils.py", line 223, in _run_worker_process
(VllmWorkerProcess pid=612122) ERROR 07-01 20:35:47 multiproc_worker_utils.py:226]     output = executor(*args, **kwargs)
(VllmWorkerProcess pid=612122) ERROR 07-01 20:35:47 multiproc_worker_utils.py:226]              ^^^^^^^^^^^^^^^^^^^^^^^^^
(VllmWorkerProcess pid=612122) ERROR 07-01 20:35:47 multiproc_worker_utils.py:226]   File "/home/hanlv/miniconda3/envs/swift/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 115, in 